# invert synthetic data using different parameterizations

## Intro : imports, prompt help

In [ ]:
import sys, glob, os
from srfpython import *
try:
    check_herrmann_codes()
except Exception:
    recompile_src90(yes=False)
    check_herrmann_codes()

In [ ]:
rm -rf _HerrMet* *.surf96 *.mod96

In [ ]:
ls ./*

In [ ]:
%run -i ../../srfpython/bin/HerrMet -h

## I/ Create synthetic data

In [ ]:
# -----------------
# generate a random depth model
# -----------------
ztop = np.unique(np.sort(np.random.rand(10))) * 3.
ztop[0] = 0.
vs   = np.linspace(.5, 3.3, len(ztop))
pr   = np.linspace(2.1, 1.8, len(ztop))
rh   = np.linspace(2.1, 2.8, len(ztop))
vp = np.array(pr) * np.array(vs)

dm = depthmodel_from_arrays(ztop, vp, vs, rh)
plt.figure(figsize = (4,8))
dm.show(gca())
dm.pr().show(gca(), color = "m", label="vp/vs")
gca().grid(True)
plt.legend()

dm.write96('dmtuto.mod96')

In [ ]:
# -----------------
# compute the corresponding dispersion curves
# -----------------

os.system('rm -f dmtuto.surf96')
%run -i ../../srfpython/bin/m96 \
    --disp dmtuto.mod96 \
        -RU0 .1  2. 12 "log" \
        -LU0 .1  2. 12 "log" \
        -RU1 .1  2. 12 "log" \
        -LU1 .1  2. 12 "log" \
        -RC0 .1  2. 12 "log" \
        -LC0 .1  2. 12 "log" \
        -RC1 .1  2. 12 "log" \
        -LC1 .1  2. 12 "log" \
        -save dmtuto.surf96
        
%run -i ../../srfpython/bin/s96 \
    --show dmtuto.surf96 \
        -inline        


## II/ Invert data with 3 different parameterizations (2, 3, or 4 layers)

In [ ]:
# -----------------
# clean up, prepare files for 3 inversions
# dmtuto.mod96 = synthetic model (the expected solution)
# dmtuto.surf96 = synthetic data (the data to invert)
# we generate 3 copies of dmtuto.surf96, to be used as targets by HerrMet
# -----------------
os.system('''
rm -rf _HerrMet*
mv dmtuto.surf96  2layers.surf96 #target 1
cp 2layers.surf96 3layers.surf96 #target 2
cp 2layers.surf96 4layers.surf96 #target 3
''')
for _ in glob.glob('./*'):
    print _, 


In [ ]:
# -----------------
# set target data, create temporary directories (1 per target file)
# -----------------
%run -i ../../srfpython/bin/HerrMet \
    --target ?layers.surf96 \
        -lunc .3 \
        -ot

In [ ]:
#ls -lrth ./*

In [ ]:
# show target (qc)
%run -i ../../srfpython/bin/s96 \
    --show _HerrMet_2layers/_HerrMet.target \
        -inline

# Parameterization

In [ ]:
# -----------------
# set parameterization template, with two layers (in ./_HerrMet.param)
# -----------------
%run -i ../../srfpython/bin/HerrMet \
    --param 2 3. \
        -t mZVSPRRH \
        -growing \
        -op \
    --send   
    
# the template file is no longer needed, 
# the next cells reproduce the file content in _HerrMet_*/_HerrMet.param    
os.system('rm -f ./_HerrMet.param') 

In [ ]:
# -----------------
# customize parameterization manually : 2 layers case
# -----------------
with open('_HerrMet_2layers/_HerrMet.param', "w") as fid:
    fid.write("""
#met DRHMIN = 0.0
#met DPRMAX = 0.0
#met DVPMIN = 0.0
#met DRHMAX = 5.0
#met DVPMAX = 5.0
#met DVSMAX = 5.0
#met DPRMIN = -5.0
#met PRIORTYPE = 'DVPDVSDRHDPR'
#met NLAYER = 2
#met TYPE = 'mZVSPRRH'
#met DVSMIN = 0.0
#fld KEY VINF VSUP
#unt - - -
#fmt %5s %16f %16f
       -Z1        -3.000000        -0.001000
       VS0         0.100000         3.500000
       VS1         0.100000         3.500000
       PR0         1.79             2.12
       PR1         1.79             2.12
       RH0         2.1              2.9
       RH1         2.1              2.9""")

In [ ]:
# -----------------
# customize parameterization manually : 3 layers case
# -----------------
with open('_HerrMet_3layers/_HerrMet.param', "w") as fid:
    fid.write("""
#met DRHMIN = 0.0
#met DPRMAX = 0.0
#met DVPMIN = 0.0
#met DRHMAX = 5.0
#met DVPMAX = 5.0
#met DVSMAX = 5.0
#met DPRMIN = -5.0
#met PRIORTYPE = 'DVPDVSDRHDPR'
#met NLAYER = 3
#met TYPE = 'mZVSPRRH'
#met DVSMIN = 0.0
#fld KEY VINF VSUP
#unt - - -
#fmt %5s %16f %16f
       -Z1        -2.              -0.001000
       -Z3        -3.000000        -1.
       VS0         0.100000         3.500000
       VS1         0.100000         3.500000
       VS2         0.100000         3.500000
       PR0         1.79             2.12
       PR1         1.79             2.12
       PR2         1.79             2.12       
       RH0         2.1              2.9
       RH1         2.1              2.9
       RH2         2.1              2.9""")
#%run -i ../../srfpython/bin/HerrMet \
#    --display _HerrMet_3layers \
#        -m96 dmtuto.mod96 \
#        -inline

In [ ]:
# -----------------
# customize parameterization manually : 4 layers case
# -----------------
with open('_HerrMet_4layers/_HerrMet.param', "w") as fid:
    fid.write("""
#met DRHMIN = 0.0
#met DPRMAX = 0.0
#met DVPMIN = 0.0
#met DRHMAX = 5.0
#met DVPMAX = 5.0
#met DVSMAX = 5.0
#met DPRMIN = -5.0
#met PRIORTYPE = 'DVPDVSDRHDPR'
#met NLAYER = 4
#met TYPE = 'mZVSPRRH'
#met DVSMIN = 0.0
#fld KEY VINF VSUP
#unt - - -
#fmt %5s %16f %16f
       -Z1        -1.000000        -0.001000
       -Z2        -2.000000        -0.500000
       -Z3        -3.000000        -1.000000
       VS0         0.100000         3.500000
       VS1         0.100000         3.500000
       VS2         0.100000         3.500000
       VS3         0.100000         3.500000
       PR0         1.79             2.12       
       PR1         1.79             2.12       
       PR2         1.79             2.12       
       PR3         1.79             2.12          
       RH0         2.1              2.9
       RH1         2.1              2.9
       RH2         2.1              2.9
       RH3         2.1              2.9""")

In [ ]:
# check parameterizations for all 3 parameterizations
#%run -i ../../srfpython/bin/HerrMet \
#    --display \
#        -m96 dmtuto.mod96 \
#        -inline

# check parameterizations for 2 layers only
%run -i ../../srfpython/bin/HerrMet \
    --display _HerrMet_2layers \
        -m96 dmtuto.mod96 \
        -inline

In [ ]:
# -----------------
# run all inversions (1 by 1) in restart mode
# -----------------
%run -i ../../srfpython/bin/HerrMet \
    -w 8 \
    -taskset "0-3" \
    -verbose 0 \
    --run \
        -nchain 16 \
        -nkeep 200 
print "DONE"

In [ ]:
# -----------------
# display results
# -----------------
%run -i ../../srfpython/bin/HerrMet \
    -verbose 0 \
    --display _HerrMet_2layers _HerrMet_3layers _HerrMet_4layers \
        -top 0. 200 1 \
        -overdisp \
        -cmap gray \
        -pdf  \
        -m96 dmtuto.mod96 \
        -inline

In [ ]:
#clean up
os.system('rm -rf *surf96 *mod96 _HerrMet*')

In [ ]:
ls ./*